# * Ex-T Revenue & Subs

## Parameter

In [31]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [32]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, REMARK, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM (
        SELECT TM_KEY_DAY
            , CASE 	WHEN METRIC_CD LIKE 'TB%S%' 
                    THEN ROW_NUMBER() OVER (PARTITION BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, AREA_CD ORDER BY TM_KEY_DAY DESC)
                    ELSE 1 END FLAG
            , CASE WHEN METRIC_CD LIKE 'TB%S%' THEN 'SUB' ELSE 'REV' END AGG_TYPE
            , METRIC_CD, METRIC_NAME, ACTUAL, COMP_CD, VERSION, AREA_CD, AREA_DESC, AREA_TYPE, LOAD_DATE, REMARK, SRC
        FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
        WHERE REMARK IN ('salisa', 'jutar11')
        AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
        AND EXISTS (
            SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
            FROM (
                SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            ) TMP
            WHERE TMP_CD = AREA_CD )
        AND TM_KEY_DAY >= {v_start_date} 
    ) TMP
    WHERE FLAG = 1
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, SUBSTR(TM_KEY_DAY,1,6)
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101



TDMDBPR : Connected

   -> Execute query... 2025-04-03, 10:58:12

   -> Summany DataFrame : 432 rows, 17 columns

TDMDBPR : Disconnected


In [33]:
''' Current Month Revenue '''

curr_mth = '202503'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_overall_df = chk_src_df.copy()
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_overall_df = src_t_rev_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_overall_df = src_t_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_overall_df[col] = src_t_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-04-02 04:40:16,20250301,20250330,0,0,"2,415,582,347","2,415,571,781","2,415,571,781",0,0,4,176,5280
1,TB2R000100,Postpaid Revenue,2025-04-02 04:20:12,20250302,20250331,0,"3,693,617,341","3,693,458,317","3,693,456,522","3,693,456,522",0,0,5,177,1770
2,TB2R010100,Postpaid Revenue B2C,2025-04-02 04:20:57,20250302,20250331,0,"3,323,334,681","3,323,260,516","3,323,258,721","3,323,258,721",0,0,5,177,1770
3,TB2R020100,Postpaid Revenue B2B,2025-04-02 04:21:10,20250302,20250331,0,"370,282,660","370,197,801","370,197,801","370,197,801",0,0,5,177,1662
4,TB3R000100,Broadband Revenue,2025-04-02 05:01:45,20250302,20250328,0,0,"1,611,388,105","1,611,388,105","1,611,388,105",0,0,4,176,1584
5,TB4R000100,TVS Revenue,2025-04-02 05:12:23,20250302,20250331,0,0,"206,989,868","207,058,092","207,015,925",0,0,4,176,1760


In [34]:
''' Average Revenue per Month '''

avg_range_start = '202401'
avg_range_end = '202503'
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_avg_mth_df = chk_src_df.copy()
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_avg_mth_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_avg_mth_df[col] = src_t_rev_avg_mth_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_avg_mth_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-04-02 04:40:16,20240101,20250330,0,0,"2,336,747,596","2,336,717,116","2,336,717,116",0,0,4,176,80080
1,TB2R000100,Postpaid Revenue,2025-04-02 04:20:12,20240102,20250331,0,"3,701,929,698","3,672,895,850","3,672,881,070","3,672,881,070",0,0,5,177,26373
2,TB2R010100,Postpaid Revenue B2C,2025-04-02 04:20:57,20240102,20250331,0,"3,295,600,849","3,289,620,667","3,289,607,033","3,289,607,938",0,0,5,177,26365
3,TB2R020100,Postpaid Revenue B2B,2025-04-02 04:21:10,20240102,20250331,0,"406,328,850","383,275,183","383,274,037","383,274,037",0,0,5,177,26265
4,TB3R000100,Broadband Revenue,2025-04-02 05:01:45,20240102,20250328,0,0,"1,575,771,226","1,575,669,903","1,575,641,848",0,0,4,176,23760
5,TB4R000100,TVS Revenue,2025-04-02 05:12:23,20240102,20250331,0,0,"320,742,178","315,431,172","319,487,295",0,0,5,1103,35374


In [35]:
''' Current Month Subscription '''

curr_mth = '202503'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH']==curr_mth]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-04-02 04:51:06,20250331,20250331,0,0,"17,382,673","17,382,673","17,382,673",0,0,4,176,176
1,TB1S000600,Prepay Active Caller Subs,2025-03-16 04:50:17,20250314,20250314,0,"14,567,698","14,355,888","14,355,888","14,355,888",0,0,5,177,177
2,TB2S000500,Postpaid Active Subs,2025-04-02 05:02:10,20250331,20250331,0,"10,226,519","10,225,976","10,225,964","10,225,964",0,0,5,177,177
3,TB2S010500,Postpaid Active Subs B2C,2025-04-02 05:02:57,20250331,20250331,0,"9,032,909","9,032,649","9,032,637","9,032,637",0,0,5,177,177
4,TB2S020400,Postpaid Active Subs B2B,2025-04-02 05:04:03,20250331,20250331,0,"1,193,610","1,193,327","1,193,327","1,193,327",0,0,5,177,177
5,TB3S000500,Broadband Active Subs,2025-04-01 04:55:31,20250331,20250331,0,0,"3,127,140","3,127,140","3,127,140",0,0,4,176,176
6,TB4S000500,TVS Active Subs,2025-04-01 04:56:24,20250331,20250331,0,0,"1,153,137","1,153,137","1,153,137",0,0,4,176,176


In [36]:
''' Average Subscription per Month '''

avg_range_start = '202401'
avg_range_end = '202503'
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-04-02 04:51:06,20240131,20250331,0,0,"17,643,728","17,643,547","17,643,547",0,0,4,176,2640
1,TB1S000600,Prepay Active Caller Subs,2025-03-16 04:50:17,20240131,20250314,0,"14,449,469","14,343,491","14,361,615","14,361,641",0,0,5,177,2655
2,TB2S000500,Postpaid Active Subs,2025-04-02 05:02:10,20240131,20250331,0,"10,035,342","9,942,043","9,942,292","9,942,314",0,0,5,177,2655
3,TB2S010500,Postpaid Active Subs B2C,2025-04-02 05:02:57,20240131,20250331,0,"8,901,018","8,897,297","8,897,492","8,897,515",0,0,5,177,2655
4,TB2S020400,Postpaid Active Subs B2B,2025-04-02 05:04:03,20240131,20250331,0,"1,134,325","1,044,746","1,044,799","1,044,799",0,0,5,177,2655
5,TB3S000500,Broadband Active Subs,2025-04-01 04:55:31,20240131,20250331,0,0,"3,058,778","3,059,992","3,059,992",0,0,4,176,2624
6,TB4S000500,TVS Active Subs,2025-04-01 04:56:24,20240131,20250331,0,0,"1,215,679","1,215,533","1,215,533",0,0,4,176,2640


In [37]:
# ''' METRIC Summary '''

# src_t_rev_mth_df = chk_src_df.copy()
# src_t_rev_mth_df = src_t_rev_mth_df.groupby(['REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
# src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD']).reset_index()

# mod_col_list = src_t_rev_mth_df.iloc[:, 6:7].columns.tolist()
# for col in mod_col_list:
#     src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_rev_mth_df.to_string(max_cols=10)}')

In [38]:
# ''' METRIC Current Month '''

# # v_tm_key_mth = '202502'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
# src_t_rev_mth_df_cols = ['REMARK', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
# print(f'\nParameter input...\n')
# print(f'   -> v_tm_key_mth: {curr_mth}')

# src_t_rev_mth_df = chk_src_df[src_t_rev_mth_df_cols].copy()
# src_t_rev_mth_df = src_t_rev_mth_df.loc[src_t_rev_mth_df['TM_KEY_MTH']==curr_mth]
# src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

# mod_col_list = src_t_rev_mth_df.iloc[:, 7:14].columns.tolist()
# for col in mod_col_list:
#     src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_rev_mth_df.to_string(max_cols=17)}')

## Actual : Transaction
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [39]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+ PARALLEL(8)*/ 
         REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    --WHERE METRIC_CD = 'TB3R000100' --TOL Revenue
    WHERE METRIC_CD IN (
        'TB1R000100' --'Prepay Revenue
        , 'TB2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         )
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Daily DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101



TDMDBPR : Connected

   -> Execute query... 2025-04-03, 10:58:15

   -> Daily DataFrame : 174 rows, 12 columns

TDMDBPR : Disconnected


In [40]:
''' METRIC by Day '''

v_tm_key_day = 20250101
# src_t_rev_day_df = ['SRC', 'REMARK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT', 'LOAD_DATE']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

# src_t_rev_day_df = chk_src_df[src_t_rev_day_df_cols].copy()
src_t_rev_day_df = chk_src_df.copy()
src_t_rev_day_df = src_t_rev_day_df.loc[src_t_rev_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_t_rev_day_df = src_t_rev_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_t_rev_day_df.iloc[:, 4:11].columns.tolist()
for col in mod_col_list:
    src_t_rev_day_df[col] = src_t_rev_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_t_rev_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_t_rev_day_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_t_rev_day_df...

   -> DataFrame : 174 rows, 12 columns


In [41]:
''' TB1R000100 : Prepay Revenue '''

v_metric_cd = 'TB1R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB1R000100 : Prepay Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1R000100_day_df = chk_src_df.copy()
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['METRIC_CD']==v_metric_cd]
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1R000100_day_df = TB1R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB1R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB1R000100_day_df[col] = TB1R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1R000100_day_df


TB1R000100 : Prepay Revenue...

   -> v_metric_cd: TB1R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB1R000100,Prepay Revenue,20250301,2025-04-02 04:40:16,0,0,"98,888,800","98,887,816","98,887,816",0,0
1,jutar11,TB1R000100,Prepay Revenue,20250302,2025-04-02 04:40:16,0,0,"97,909,469","97,909,227","97,909,227",0,0
2,jutar11,TB1R000100,Prepay Revenue,20250303,2025-04-02 04:40:16,0,0,"97,338,456","97,337,930","97,337,930",0,0
3,jutar11,TB1R000100,Prepay Revenue,20250304,2025-04-02 04:40:16,0,0,"87,083,889","87,083,410","87,083,410",0,0
4,jutar11,TB1R000100,Prepay Revenue,20250305,2025-04-02 04:40:16,0,0,"88,600,809","88,600,652","88,600,652",0,0
5,jutar11,TB1R000100,Prepay Revenue,20250306,2025-04-02 04:40:16,0,0,"83,013,360","83,013,234","83,013,234",0,0
6,jutar11,TB1R000100,Prepay Revenue,20250307,2025-04-02 04:40:16,0,0,"84,166,268","84,165,036","84,165,036",0,0
7,jutar11,TB1R000100,Prepay Revenue,20250308,2025-04-02 04:40:16,0,0,"79,922,203","79,922,121","79,922,121",0,0
8,jutar11,TB1R000100,Prepay Revenue,20250309,2025-04-02 04:40:16,0,0,"77,198,101","77,198,051","77,198,051",0,0
9,jutar11,TB1R000100,Prepay Revenue,20250310,2025-04-02 04:40:16,0,0,"83,599,377","83,599,259","83,599,259",0,0


In [42]:
''' TB2R010100 : Postpaid Revenue B2C '''

v_metric_cd = 'TB2R010100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB2R010100 : Postpaid Revenue B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB2R010100_day_df = chk_src_df.copy()
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['METRIC_CD']==v_metric_cd]
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB2R010100_day_df = TB2R010100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB2R010100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB2R010100_day_df[col] = TB2R010100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB2R010100_day_df


TB2R010100 : Postpaid Revenue B2C...

   -> v_metric_cd: TB2R010100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB2R010100,Postpaid Revenue B2C,20250302,2025-04-02 04:20:57,0,"475,677,107","475,669,335","475,668,436","475,668,436",0,0
1,jutar11,TB2R010100,Postpaid Revenue B2C,20250306,2025-04-02 04:20:57,0,"280,064,853","280,059,508","280,059,508","280,059,508",0,0
2,jutar11,TB2R010100,Postpaid Revenue B2C,20250310,2025-04-02 04:20:57,0,"442,367,545","442,358,394","442,358,394","442,358,394",0,0
3,jutar11,TB2R010100,Postpaid Revenue B2C,20250313,2025-04-02 04:20:57,0,"389,323,269","389,313,539","389,313,090","389,313,090",0,0
4,jutar11,TB2R010100,Postpaid Revenue B2C,20250316,2025-04-02 04:20:57,0,"353,869,256","353,863,690","353,863,284","353,863,284",0,0
5,jutar11,TB2R010100,Postpaid Revenue B2C,20250319,2025-04-02 04:20:57,0,"352,200,233","352,190,931","352,190,931","352,190,931",0,0
6,jutar11,TB2R010100,Postpaid Revenue B2C,20250322,2025-04-02 04:20:57,0,"360,529,920","360,521,563","360,521,523","360,521,523",0,0
7,jutar11,TB2R010100,Postpaid Revenue B2C,20250325,2025-04-02 04:20:57,0,"326,526,069","326,517,424","326,517,424","326,517,424",0,0
8,jutar11,TB2R010100,Postpaid Revenue B2C,20250328,2025-04-02 04:20:57,0,"342,677,592","342,667,294","342,667,294","342,667,294",0,0
9,jutar11,TB2R010100,Postpaid Revenue B2C,20250331,2025-04-01 04:20:43,0,"98,838","98,838","98,838","98,838",0,0


In [43]:
''' TB3R000100 : Broadband Revenue '''

v_metric_cd = 'TB3R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB3R000100 : Broadband Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD']==v_metric_cd]
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TB3R000100 : Broadband Revenue...

   -> v_metric_cd: TB3R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB3R000100,Broadband Revenue,20250302,2025-04-02 05:01:45,0,0,"217,475,728","217,475,728","217,475,728",0,0
1,salisa,TB3R000100,Broadband Revenue,20250306,2025-04-02 05:01:45,0,0,"142,944,002","142,944,002","142,944,002",0,0
2,salisa,TB3R000100,Broadband Revenue,20250310,2025-04-02 05:01:45,0,0,"232,324,981","232,324,981","232,324,981",0,0
3,salisa,TB3R000100,Broadband Revenue,20250313,2025-04-02 05:01:45,0,0,"172,673,270","172,673,270","172,673,270",0,0
4,salisa,TB3R000100,Broadband Revenue,20250316,2025-04-02 05:01:45,0,0,"173,661,847","173,661,847","173,661,847",0,0
5,salisa,TB3R000100,Broadband Revenue,20250319,2025-04-02 05:01:45,0,0,"166,391,300","166,391,300","166,391,300",0,0
6,salisa,TB3R000100,Broadband Revenue,20250322,2025-04-02 05:01:45,0,0,"166,821,225","166,821,225","166,821,225",0,0
7,salisa,TB3R000100,Broadband Revenue,20250325,2025-04-02 05:01:45,0,0,"165,423,292","165,423,292","165,423,292",0,0
8,salisa,TB3R000100,Broadband Revenue,20250328,2025-04-02 05:01:45,0,0,"173,672,460","173,672,460","173,672,460",0,0


In [44]:
''' TB4R000100 : TVS Revenue '''

v_metric_cd = 'TB4R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB4R000100 : TVS Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD']==v_metric_cd]
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TB4R000100 : TVS Revenue...

   -> v_metric_cd: TB4R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB4R000100,TVS Revenue,20250302,2025-04-02 05:12:23,0,0,"34,083,992","34,076,333","34,087,805",0,0
1,salisa,TB4R000100,TVS Revenue,20250306,2025-04-02 05:12:23,0,0,"9,579,554","9,658,492","9,654,308",0,0
2,salisa,TB4R000100,TVS Revenue,20250310,2025-04-02 05:12:23,0,0,"29,690,369","29,774,562","29,787,584",0,0
3,salisa,TB4R000100,TVS Revenue,20250313,2025-04-02 05:12:23,0,0,"21,810,483","21,856,803","21,854,114",0,0
4,salisa,TB4R000100,TVS Revenue,20250316,2025-04-02 05:12:23,0,0,"23,606,171","23,623,382","23,616,942",0,0
5,salisa,TB4R000100,TVS Revenue,20250319,2025-04-02 05:12:23,0,0,"20,526,396","20,538,580","20,538,484",0,0
6,salisa,TB4R000100,TVS Revenue,20250322,2025-04-02 05:12:23,0,0,"20,876,418","20,882,618","20,874,975",0,0
7,salisa,TB4R000100,TVS Revenue,20250325,2025-04-02 05:12:23,0,0,"21,261,482","21,261,667","21,257,234",0,0
8,salisa,TB4R000100,TVS Revenue,20250328,2025-04-02 05:12:23,0,0,"22,958,239","23,020,841","23,020,582",0,0
9,salisa,TB4R000100,TVS Revenue,20250331,2025-04-02 05:12:23,0,0,"2,596,765","2,364,815","2,323,898",0,0


## Issue
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [46]:
''' Execute Issue by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+ PARALLEL(8)*/ 
         REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    WHERE METRIC_CD = 'TB1S000600' --Prepay Active Caller Subs
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Issue DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Issue DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101

TDMDBPR : Connected

   -> Execute query... 2025-04-03, 11:02:51

   -> Issue DataFrame : 73 rows, 12 columns

TDMDBPR : Disconnected


In [48]:
''' TB1S000600 : Prepay Active Caller Subs '''

v_metric_cd = 'TB1S000600'
# v_tm_key_day = int(str(curr_mth)+str('01'))
v_tm_key_day = 20250101
print(f'\nTB1S000600 : Prepay Active Caller Subs...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1S000600_day_df = chk_src_df.copy()
TB1S000600_day_df = TB1S000600_day_df.loc[TB1S000600_day_df['METRIC_CD']==v_metric_cd]
TB1S000600_day_df = TB1S000600_day_df.loc[TB1S000600_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1S000600_day_df = TB1S000600_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB1S000600_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB1S000600_day_df[col] = TB1S000600_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1S000600_day_df


TB1S000600 : Prepay Active Caller Subs...

   -> v_metric_cd: TB1S000600
   -> v_tm_key_day: 20250101


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB1S000600,Prepay Active Caller Subs,20250101,2025-02-13 20:22:42,0,"14,500,948","14,255,773","14,390,664","14,390,664",0,0
1,jutar11,TB1S000600,Prepay Active Caller Subs,20250102,2025-02-13 20:22:42,0,"14,566,283","14,329,701","14,457,707","14,457,707",0,0
2,jutar11,TB1S000600,Prepay Active Caller Subs,20250103,2025-02-13 20:22:42,0,"14,569,051","14,333,472","14,460,859","14,460,859",0,0
3,jutar11,TB1S000600,Prepay Active Caller Subs,20250104,2025-02-13 20:22:42,0,"14,561,774","14,328,812","14,456,436","14,456,436",0,0
4,jutar11,TB1S000600,Prepay Active Caller Subs,20250105,2025-02-13 20:22:42,0,"14,562,484","14,331,937","14,459,451","14,459,451",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
68,jutar11,TB1S000600,Prepay Active Caller Subs,20250310,2025-03-16 04:50:17,0,"14,598,050","14,420,153","14,420,153","14,420,153",0,0
69,jutar11,TB1S000600,Prepay Active Caller Subs,20250311,2025-03-16 04:50:17,0,"14,576,017","14,396,580","14,396,580","14,396,580",0,0
70,jutar11,TB1S000600,Prepay Active Caller Subs,20250312,2025-03-16 04:50:17,0,"14,567,955","14,384,295","14,384,295","14,384,295",0,0
71,jutar11,TB1S000600,Prepay Active Caller Subs,20250313,2025-03-16 04:50:17,0,"14,556,263","14,369,806","14,369,806","14,369,806",0,0
